# GPT2

In [2]:
dataset = ["""{
    "content": "世锦赛的整体水平远高于亚洲杯，要如同亚洲杯那样“鱼与熊掌兼得”，就需要各方面密切配合、#idiom#。作为主帅的俞觉敏，除了得打破保守思想，敢于破格用人，还得巧于用兵、#idiom#、灵活排阵，指挥得当，力争通过比赛推新人、出佳绩、出新的战斗力。",
    "realCount": 2,
    "groundTruth": ["通力合作", "有的放矢"],
    "candidates": [
        ["凭空捏造", "高头大马", "通力合作", "同舟共济", "和衷共济", "蓬头垢面", "紧锣密鼓"],
        ["叫苦连天", "量体裁衣", "金榜题名", "百战不殆", "知彼知己", "有的放矢", "风流才子"]
    ]
}"""]
s = "世锦赛的整体水平远高于亚洲杯，要如同亚洲杯那样“鱼与熊掌兼得”，就需要各方面密切配合、通力合作。作为主帅的俞觉敏，除了得打破保守思想，敢于破格用人，还得巧于用兵、</s>、灵活排阵，指挥得当，力争通过比赛推新人、出佳绩、出新的战斗力。可选成语有：叫苦连天 | 量体裁衣 | 金榜题名 | 百战不殆 | 知彼知己 | 风流才子，请选出最佳的一项。"

In [3]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


In [44]:
import os
import numpy as np
import torch
import json
import re
from torch import cuda, nn, optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import GPT2LMHeadModel, BertTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [5]:
!pip3 install wandb

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


In [6]:
import wandb
wandb.login()
wandb.init(
    project = 'zootopia',
    config = {
        'epochs': 5
        # 'learning_rate': lr,
    }
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


# Load model

In [7]:
# T5
# tokenizer = AutoTokenizer.from_pretrained("mxmax/Chinese_Chat_T5_Base")
# model = AutoModelForSeq2SeqLM.from_pretrained("mxmax/Chinese_Chat_T5_Base") 

# device = 'cuda' if cuda.is_available() else 'cpu'
# model.to(device)

In [8]:
# GPT2
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall", cache_dir="./models")
model = GPT2LMHeadModel.from_pretrained("uer/gpt2-chinese-cluecorpussmall", cache_dir="./models")

device = 'cuda' if cuda.is_available() else 'cpu'
model = model.to(device)

# Fine-tuning model 

## Data preprocessing

In [9]:
def prompt1(text, candidates, choice):
    input_text = "请从（）里选择出最合适的成语: " + text
    for i in range(len(choice)):
        candidates_str = '|'.join([c for c in candidates[i]])
        input_text = input_text.replace('#idiom#', "（" + candidates_str + "）", 1)
    return input_text

In [10]:
def prompt2(text, candidates, choice):
    input_text = text.replace('#idiom#', "_")
    input_text = "请依次从（）选择出最合适的成语填入_: " + input_text
    for i in range(len(choice)):
        candidates_str = '|'.join([c for c in candidates[i]])
        input_text = input_text.replace('（）', "（" + candidates_str + "）（）", 1)
    input_text = input_text.replace("（）", "")
    return input_text

In [11]:
def prompt3(text, candidates, choice):
    ex = '''选择：[[“凭空捏造“, “高头大马“, “通力合作“, “同舟共济“, “和衷共济“, “蓬头垢面“, “紧锣密鼓“],[“叫苦连天“, “量体裁衣“, “金榜题名“, “百战不殆“, “知彼知己“, “有的放矢“, “风流才子“]]\n输入：“世锦赛的整体水平远高于亚洲杯，要如同亚洲杯那样“鱼与熊掌兼得“，就需要各方面密切配合、#idiom#。作为主帅的俞觉敏，除了得打破保守思想，敢于破格用人，还得巧于用兵、#idiom#、灵活排阵，指挥得当，力争通过比赛推新人、出佳绩、出新的战斗力。”\n输出：通力合作, 有的放矢\n'''
    input_text = ex
    input_text = input_text + f"选择：{candidates}\n输入：{text}\n输出："
    return input_text

In [12]:
def preprocess_data(data, prompt):
    input_texts = []
    labels = []

    for example in data:
        example = json.loads(example)

        input_text = example['content']
        ground_truth = example['groundTruth']
        candidates = example['candidates']

        input_text = prompt(input_text, candidates, ground_truth)
        if len(input_text) > 500:
            continue
        input_texts.append(input_text)
        labels.append('、'.join(ground_truth))

    # inputs = tokenizer(input_texts, return_token_type_ids=False)
    # labels = tokenizer(labels, return_token_type_ids=False)
    # return inputs, labels
    concat_inputs = tokenizer(input_texts, labels, return_token_type_ids=False)
    return concat_inputs

In [13]:
# Load the Chinese Idioms dataset
train_data_file = './data/train_15000.txt'
val_data_file = './data/dev_2000.txt'


with open(train_data_file) as f:
    train_data = f.readlines()

with open(val_data_file) as f:
    val_data = f.readlines()
    
train_inputs = preprocess_data(train_data, prompt2)
val_inputs = preprocess_data(val_data, prompt2)

* Notes:
type_ids, offsets, special_tokens_mask and overflowing are Not available in GPT-2

In [14]:
print("Input length:", len(train_inputs))
print(" ")
print("Input example:\n", tokenizer.decode(train_inputs['input_ids'][0]))
print(" ")
print("Input ID example:\n", train_inputs['input_ids'][0])
print(" ")
print("Tokens:\n", [tokenizer.convert_ids_to_tokens(id) for id in train_inputs['input_ids'][0]])
print(" ")
print("Attention Mask:", train_inputs['attention_mask'][0])

Input length: 2
 
Input example:
 [CLS] 请 依 次 从 （ 超 凡 入 圣 | 骨 瘦 如 柴 | 青 面 獠 牙 | 虎 背 熊 腰 | 成 人 之 美 | 肥 头 大 耳 | 神 不 守 舍 ） 选 择 出 最 合 适 的 成 语 填 入 _ : _ 的 掌 柜 只 穿 一 件 衬 衫 ， 坐 在 柜 台 里 。 几 个 堂 倌 穿 着 脏 得 发 黑 的 白 工 作 服 ， 因 为 没 有 顾 客 ， 都 散 坐 在 桌 子 旁 。 这 当 儿 看 到 这 位 不 寻 常 的 客 人 ， 都 露 出 好 奇 的 神 色 列 宁 曾 批 评 他 理 论 上 的 错 误 ， 同 时 认 为 他 [UNK] 所 写 的 全 部 哲 学 ， 赶 紧 迎 上 前 来 伺 候 。 聂 赫 留 朵 夫 要 了 一 瓶 矿 泉 水 ， 在 离 窗 较 远 的 地 方 挨 着 一 张 铺 有 肮 脏 桌 布 的 小 桌 坐 下 。 [SEP] 肥 头 大 耳 [SEP]
 
Input ID example:
 [101, 6435, 898, 3613, 794, 8020, 6631, 1127, 1057, 1760, 170, 7755, 4607, 1963, 3395, 170, 7471, 7481, 4358, 4280, 170, 5988, 5520, 4220, 5587, 170, 2768, 782, 722, 5401, 170, 5503, 1928, 1920, 5455, 170, 4868, 679, 2127, 5650, 8021, 6848, 2885, 1139, 3297, 1394, 6844, 4638, 2768, 6427, 1856, 1057, 142, 131, 142, 4638, 2958, 3385, 1372, 4959, 671, 816, 6137, 6136, 8024, 1777, 1762, 3385, 1378, 7027, 511, 1126, 702, 1828, 944, 4959, 4708, 5552, 2533, 1355, 7946, 4638, 4635, 2339, 868, 3302, 8024, 1728, 711, 3766, 3300

### IdiomDataset

In [15]:
class IdiomDataset(Dataset):

    def __init__(self, data) -> None:
        super().__init__()
        self.data = data
        self.label = self._get_label(data["input_ids"])
        self.is_inference = False
    
    def __len__(self):
        return len(self.data["input_ids"])
    
    def _get_label(self, inputs):
        labels = []
        for inp in inputs:
            sep_idx = inp.index(102)
            label = [-100] * len(inp)
            label[sep_idx + 1:] = inp[sep_idx + 1:]
            labels.append(label)
        return labels

    def inference(self):
        self.is_inference = True

    def train(self):
        self.is_inference = False
        
    def is_inference(self):
        return self.is_inference

    def __getitem__(self, index):
        if not self.is_inference:
            return {"input_ids": self.data["input_ids"][index], "attention_mask": self.data["attention_mask"][index], "labels": self.label[index]}
        else:
            sep_idx = self.data["input_ids"][index].index(102)
            input_ids = self.data["input_ids"][index][:sep_idx+1]
            att_mask = self.data["attention_mask"][index][:sep_idx+1]
            label = self.label[index][sep_idx+1:]
            return {"input_ids": input_ids, "attention_mask":att_mask, "labels":label}


def collate_fn(batch):
    batch_input_ids = [torch.LongTensor(each["input_ids"]) for each in batch]
    batch_att_mask = [torch.LongTensor(each["attention_mask"]) for each in batch]
    batch_label = [torch.LongTensor(each["labels"]) for each in batch]
    padded_batch_input_ids = pad_sequence(batch_input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    padded_batch_att_mask = pad_sequence(batch_att_mask, batch_first=True, padding_value=0)
    padded_batch_label = pad_sequence(batch_label, batch_first=True, padding_value=-100)
    return {"input_ids": padded_batch_input_ids, "attention_mask": padded_batch_att_mask, "labels": padded_batch_label}

def to_device(data, device):
    new_data = {}
    for k in data:
        new_data[k] = data[k].to(device)
    return new_data

In [16]:
train_dataset = IdiomDataset(train_inputs)
train_loader = DataLoader(train_dataset, batch_size=8, collate_fn=collate_fn, shuffle=True)

val_dataset = IdiomDataset(val_inputs)
val_loader = DataLoader(val_dataset, batch_size=8, collate_fn=collate_fn, shuffle=False)

## Train model

In [42]:
def train(model:nn.Module, train_loader:DataLoader, optimizer:optim.Optimizer, log_step=100):
    model.train()
    epoch_loss = 0.0
    log_loss = 0.0
    for idx, batch in enumerate(train_loader, 1):
        optimizer.zero_grad()
        batch = to_device(batch, device)
        loss = model(**batch).loss
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        log_loss += loss.item()
        
        wandb.log({"batch": idx, "train loss": loss.item()})
        wandb.log({"batch": idx, "acc train loss": log_loss})
        
        if idx % log_step == 0:
            print(f"Train Step: {idx} Loss: {log_loss / log_step}")
            log_loss = 0.0
    return epoch_loss / len(train_loader)
        

@torch.no_grad()
def evaluate(model:nn.Module, eval_loader:DataLoader):
    eval_loss = 0.0
    correct = 0
    total = 0
    model.eval()
    for batch in eval_loader:
        batch = to_device(batch, device)
        output = model(**batch)
        loss = output.loss
        eval_loss += loss.item()
        pred = output.logits.argmax(-1)[..., :-1]
        label = batch["labels"][..., 1:]
        correct += torch.where(label!=-100, pred==label, 0).sum().item()
        total += torch.sum(label != -100).item()

    eval_acc = correct / total
    eval_loss = eval_loss / len(eval_loader) 
    print(total, correct)
    return eval_acc, eval_loss

In [18]:
epoches = 5
optimizer = optim.Adam(model.parameters(), lr=2e-5)
model.train()

for epoch in range(1, epoches+1):
    print(f"Training Epoch {epoch}")
    train_loss = train(model, train_loader, optimizer)
    print(f"Epoch {epoch} Training Loss: {train_loss}")
    eval_acc, eval_loss = evaluate(model, val_loader)
    print(f"Epoch {epoch} Eval Acc: {eval_acc}; Eval Loss: {eval_loss}")
torch.save(model.state_dict(), "gpt2_ckpt.pt")

Training Epoch 1
Train Step: 100 Loss: 0.9758239647746086
Train Step: 200 Loss: 0.7055644583702088
Train Step: 300 Loss: 0.6545085912942886
Train Step: 400 Loss: 0.6130328238010406
Train Step: 500 Loss: 0.5863486902415752
Train Step: 600 Loss: 0.5656983011960983
Train Step: 700 Loss: 0.5777527061104775
Train Step: 800 Loss: 0.5686028027534484
Train Step: 900 Loss: 0.5461933276057244
Train Step: 1000 Loss: 0.5387009255588054
Train Step: 1100 Loss: 0.5242047330737114
Train Step: 1200 Loss: 0.5305087232589721
Train Step: 1300 Loss: 0.5185100583732128
Train Step: 1400 Loss: 0.505561883598566
Train Step: 1500 Loss: 0.5049174413084984
Train Step: 1600 Loss: 0.5270890671014786
Train Step: 1700 Loss: 0.5028500817716122
Train Step: 1800 Loss: 0.5018112400174141
Epoch 1 Training Loss: 0.5768633379856746
11980 10336
Epoch 1 Eval Acc: 0.862771285475793; Eval Loss: 0.4282213596105576
Training Epoch 2
Train Step: 100 Loss: 0.37278037771582606
Train Step: 200 Loss: 0.39200669184327125
Train Step: 300

In [19]:
# torch.save(model.state_dict(), "gpt2_ckpt.pt")

## Evaluation

In [58]:
from transformers import StoppingCriteria, StoppingCriteriaList
class KeywordsStoppingCriteria(StoppingCriteria):
    def __init__(self, keywords_ids:list):
        self.keywords = keywords_ids

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        if all((input_ids==self.keywords[0]).sum(dim=-1) >= 2):
            return True
        return False

stop_words = ["[SEP]"]
stop_ids = [tokenizer.convert_tokens_to_ids(w) for w in stop_words]
stop_criteria = KeywordsStoppingCriteria(stop_ids)
stop_criteria_list = StoppingCriteriaList([stop_criteria])


def process_sys(pred):
    extracted_idioms = []

    for generated_text in pred:
        idioms = set()
        for text in generated_text:
            matches = re.findall(r'(?:^|[SEP]\s?)[^\u4e00-\u9fff]*([\u4e00-\u9fff]{4})', text)
            idioms.update(matches)
        extracted_idioms.append(idioms)

    return extracted_idioms


@torch.no_grad()
def fill_idiom(model, loader):
    all_preds = []
    all_labels = []
    model.eval()
    for batch in loader:   
        batch = to_device(batch, device)
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        # pos_ids = batch["position_ids"]
        labels = batch["labels"]
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, return_dict_in_generate=True, pad_token_id=50256, max_length=512, top_k=10, stopping_criteria=stop_criteria_list)
        pred_start = torch.nonzero(input_ids==tokenizer.sep_token_id, as_tuple=True)[1][0] + 1
        truncated_outputs = []
        for out in outputs["sequences"]:
            sep_idxs = torch.nonzero(out==tokenizer.sep_token_id, as_tuple=True)[0]
            if len(sep_idxs) == 1:
                end_idx = -1
            else:
                end_idx = sep_idxs[1]
            truncated_outputs.append(out[pred_start:end_idx])
        decode_texts = tokenizer.batch_decode(truncated_outputs)
        gold_texts = tokenizer.batch_decode([l[l != -100][:-1] for l in labels])

        for gold, decode in zip(gold_texts, decode_texts):
            l = set(gold.replace(" ", "").split("、"))
            p = set(decode.replace(" ", "").split("、"))
            all_labels.append(l)
            all_preds.append(p)
        
        all_preds = process_sys(all_preds)
    
    return all_preds, all_labels


def f1_score(sys, gold):
    tp = 0
    t = 0
    p = 0
    for s, g in zip(sys, gold):
        t += len(g)
        p += len(s)
        tp += len(g & s)
    precision = tp / p if p != 0 else 0
    recall = tp / t if t != 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    return precision, recall, f1, tp


def left_pad_sequence(sequence, batch_first, padding_value=0):
    padded = []
    max_len = max(len(each) for each in sequence)
    for each in sequence:
        if not isinstance(each, torch.LongTensor):
            each = torch.LongTensor(each)
        pad = torch.full((max_len-len(each),), fill_value=padding_value,dtype=each.dtype)
        padded.append(torch.cat([pad, each]))
    padded = torch.vstack(padded)
    if not batch_first:
        padded = padded.permute(1, 0, 2)
    return padded
        
def inference_colate_fn(batch):
    batch_input_ids = [torch.LongTensor(each["input_ids"]) for each in batch]
    batch_att_mask = [torch.LongTensor(each["attention_mask"]) for each in batch]
    batch_label = [torch.LongTensor(each["labels"]) for each in batch]
    batch_position_ids = [torch.arange(len(each["input_ids"]), dtype=torch.long) for each in batch]
    
    padded_batch_input_ids = left_pad_sequence(batch_input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    padded_batch_att_mask = left_pad_sequence(batch_att_mask, batch_first=True, padding_value=0)
    padded_batch_label = pad_sequence(batch_label, batch_first=True, padding_value=-100)
    # padded_batch_position_ids = left_pad_sequence(batch_position_ids, batch_first=True, padding_value=0)
    # return {"input_ids": padded_batch_input_ids, "attention_mask": padded_batch_att_mask, "position_ids":padded_batch_position_ids, "labels": padded_batch_label}   
    return {"input_ids": padded_batch_input_ids, "attention_mask": padded_batch_att_mask, "labels": padded_batch_label}    

val_dataset.inference()
inf_loader = DataLoader(val_dataset, batch_size=64, collate_fn=inference_colate_fn)

In [59]:
model.load_state_dict(torch.load("gpt2_ckpt.pt", map_location=device))
sys, gold = fill_idiom(model, loader=inf_loader)
p, r, f1, tp = f1_score(sys, gold)

print(f"Accuracy for Validation set is {f1}")
print(f"Accuracy for Validation set is {tp}")

Accuracy for Validation set is 0.467101114147572
Accuracy for Validation set is 1111


In [60]:
# Load the Chinese Idioms dataset For Test set
test_data_file = './data/test_2000.txt'

with open(test_data_file, encoding='utf-8', errors='ignore') as f:
    test_data = f.readlines()

test_inputs = preprocess_data(test_data,prompt2)
test_dataset = IdiomDataset(test_inputs)
test_loader = DataLoader(test_dataset, batch_size=8, collate_fn=collate_fn, shuffle=True)

In [61]:
# test set
model.load_state_dict(torch.load("gpt2_ckpt.pt", map_location=device))
sys, gold = fill_idiom(model, test_loader)
p, r, f1, tp = f1_score(sys, gold)

print((f"F1 score for Test set is {f1}"))
print(f"Accuracy for Test set is {tp}")

F1 score for Test set is 0.6436031331592688
Accuracy for Test set is 1479


In [62]:
sys[:10]

[{'千方百计'},
 {'摇身一变', '问题是附'},
 set(),
 {'明枪暗箭', '许太李嘉', '铜墙铁壁'},
 {'旗鼓相当', '苦思冥想', '赛第二回'},
 {'一针见血', '争做出深'},
 set(),
 set(),
 {'妙趣横生', '对答如流'},
 set()]

In [63]:
gold[:10]

[{'千方百计'},
 {'摇身一变'},
 {'流言蜚语'},
 {'明枪暗箭', '铜墙铁壁'},
 {'旗鼓相当', '苦思冥想'},
 {'一针见血'},
 {'事半功倍'},
 {'自给自足'},
 {'妙趣横生', '对答如流'},
 {'终身大事'}]

In [65]:
print(tokenizer.decode([12]))
tokenizer.decode([12, 31200, 2057, 1498, 1278, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

[unused12]


'[unused12] [UNK] 媞 咽 医 [unused1] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

# Prompt engineering

## Load model

In [66]:
# T5
# tokenizer = AutoTokenizer.from_pretrained("mxmax/Chinese_Chat_T5_Base")
# model = AutoModelForSeq2SeqLM.from_pretrained("mxmax/Chinese_Chat_T5_Base") 

# device = 'cuda' if cuda.is_available() else 'cpu'
# model.to(device)

In [67]:
# GPT2
# tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall", cache_dir="./models")
# model = GPT2LMHeadModel.from_pretrained("uer/gpt2-chinese-cluecorpussmall", cache_dir="./models")

# device = 'cuda' if cuda.is_available() else 'cpu'
# model = model.to(device)

## Data processing

In [68]:
def prompt1(text, candidates, choice):
    input_text = "请从（）里选择出最合适的成语: " + text
    for i in range(len(choice)):
        candidates_str = '|'.join([c for c in candidates[i]])
        input_text = input_text.replace('#idiom#', "（" + candidates_str + "）", 1) 
    return input_text

In [69]:
def prompt2(text, candidates, choice):
    input_text = "请依次从（）选择出最合适的成语填入#idiom#: " + text
    for i in range(len(choice)):
        candidates_str = '|'.join([c for c in candidates[i]])
        input_text = input_text.replace('（）', "（" + candidates_str + "）（）", 1)
    input_text = input_text.replace("（）", "")
    return input_text

In [70]:
def prompt3(text, candidates, choice):
    ex = '''选择：[[“凭空捏造“, “高头大马“, “通力合作“, “同舟共济“, “和衷共济“, “蓬头垢面“, “紧锣密鼓“],[“叫苦连天“, “量体裁衣“, “金榜题名“, “百战不殆“, “知彼知己“, “有的放矢“, “风流才子“]]\n输入：“世锦赛的整体水平远高于亚洲杯，要如同亚洲杯那样“鱼与熊掌兼得“，就需要各方面密切配合、#idiom#。作为主帅的俞觉敏，除了得打破保守思想，敢于破格用人，还得巧于用兵、#idiom#、灵活排阵，指挥得当，力争通过比赛推新人、出佳绩、出新的战斗力。”\n输出：通力合作, 有的放矢\n'''
    input_text = ex
    input_text = input_text + f"选择：{candidates}\n输入：{text}\n输出："
    return input_text

In [71]:
def preprocess_data(data, prompt):
    input_texts = []
    labels = []

    for example in data:
        example = json.loads(example)

        input_text = example['content']
        ground_truth = example['groundTruth']
        candidates = example['candidates']

        input_text = prompt(input_text, candidates, ground_truth)
        if len(input_text) > 500:
            continue
        input_texts.append(input_text)
        labels.append('、'.join(ground_truth))

    # inputs = tokenizer(input_texts, return_token_type_ids=False)
    # labels = tokenizer(labels, return_token_type_ids=False)
    # return inputs, labels
    concat_inputs = tokenizer(input_texts, labels, return_token_type_ids=False)
    return concat_inputs

In [72]:
# Load the Chinese Idioms dataset For Test set
test_data_file = './data/test_2000.txt'
with open(test_data_file, encoding='utf-8', errors='ignore') as f:
    test_data = f.readlines()

In [73]:
test_inputs1 = preprocess_data(test_data, prompt1)
test_dataset1 = IdiomDataset(test_inputs1)
test_loader1 = DataLoader(test_dataset1, batch_size=8, collate_fn=collate_fn, shuffle=True)

In [74]:
test_inputs2 = preprocess_data(test_data, prompt2)
test_dataset2 = IdiomDataset(test_inputs2)
test_loader2 = DataLoader(test_dataset2, batch_size=8, collate_fn=collate_fn, shuffle=True)

In [75]:
test_inputs3 = preprocess_data(test_data, prompt3)
test_dataset3 = IdiomDataset(test_inputs3)
test_loader3 = DataLoader(test_dataset3, batch_size=8, collate_fn=collate_fn, shuffle=True)

In [76]:
print([tokenizer.convert_ids_to_tokens(id) for id in test_inputs2['input_ids'][0]])
print([tokenizer.convert_ids_to_tokens(id) for id in test_inputs3['input_ids'][0]])

['[CLS]', '请', '依', '次', '从', '（', '旷', '日', '持', '久', '|', '公', '正', '廉', '洁', '|', '苦', '口', '婆', '心', '|', '现', '身', '说', '法', '|', '白', '日', '做', '梦', '|', '深', '入', '浅', '出', '|', '肺', '腑', '之', '言', '）', '选', '择', '出', '最', '合', '适', '的', '成', '语', '填', '入', '#', 'id', '##io', '##m', '#', ':', '只', '要', '路', '过', '的', '旅', '客', '稍', '有', '迟', '疑', '，', '或', '者', '对', '他', '们', '的', '宣', '传', '单', '多', '看', '几', '眼', '，', '基', '本', '上', '这', '个', '旅', '客', '就', '别', '想', '轻', '松', '脱', '身', '了', '，', '记', '者', '就', '在', '9', '月', '3', '日', '接', '站', '时', '目', '睹', '了', '这', '样', '一', '幕', '：', '一', '个', '学', '生', '接', '过', '招', '生', '人', '员', '递', '来', '的', '宣', '传', '单', '，', '只', '是', '问', '了', '一', '下', '[UNK]', '你', '们', '学', '校', '有', '没', '有', '分', '数', '要', '求', '？', '[UNK]', '两', '个', '招', '生', '人', '员', '就', '[UNK]', '白', '话', '[UNK]', '开', '了', '，', '一', '个', '表', '示', '分', '数', '都', '好', '说', '，', '只', '要', '有', '好', '学', '的', '精', '神', '；', '另', '一', '个', '则', '#', 'id

### Prompt Engineering with fine-tuning

In [77]:
model.load_state_dict(torch.load("gpt2_ckpt.pt", map_location=device))

# prompt engineering for test set (prompt1)
sys1, gold1 = fill_idiom(model, test_loader1)
p1, r1, f11, tp1 = f1_score(sys1, gold1)
print((f"F1 score for Test set is {f11}"))
print(f"Accuracy for Test set is {tp1}")

# prompt engineering for test set (prompt2)
sys2, gold2 = fill_idiom(model, test_loader2)
p2, r2, f12, tp2 = f1_score(sys2, gold2)
print((f"F1 score for Test set is {f12}"))
print(f"Accuracy for Test set is {tp2}")

# prompt engineering for test set (prompt3)
sys3, gold3 = fill_idiom(model, test_loader3)
p3, r3, f13, tp3 = f1_score(sys3, gold3)
print((f"F1 score for Test set is {f13}"))
print(f"Accuracy for Test set is {tp3}")

F1 score for Test set is 0.6443495843630644
Accuracy for Test set is 1434
F1 score for Test set is 0.6355099366673946
Accuracy for Test set is 1455
F1 score for Test set is 0.6471647164716471
Accuracy for Test set is 719


### Prompt Engineering without fine-tuning

In [78]:
# GPT2
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall", cache_dir="./models")
model = GPT2LMHeadModel.from_pretrained("uer/gpt2-chinese-cluecorpussmall", cache_dir="./models")

device = 'cuda' if cuda.is_available() else 'cpu'
model = model.to(device)

In [79]:
# prompt engineering for test set (prompt1)
sys1, gold1 = fill_idiom(model, test_loader1)
p1, r1, f11, tp1 = f1_score(sys1, gold1)
print((f"F1 score for Test set is {f11}"))
print(f"Accuracy for Test set is {tp1}")

# prompt engineering for test set (prompt2)
sys2, gold2 = fill_idiom(model, test_loader2)
p2, r2, f12, tp2 = f1_score(sys2, gold2)
print((f"F1 score for Test set is {f12}"))
print(f"Accuracy for Test set is {tp2}")

# prompt engineering for test set (prompt3)
sys3, gold3 = fill_idiom(model, test_loader3)
p3, r3, f13, tp3 = f1_score(sys3, gold3)
print((f"F1 score for Test set is {f13}"))
print(f"Accuracy for Test set is {tp3}")

F1 score for Test set is 0.6149796602057909
Accuracy for Test set is 1285
F1 score for Test set is 0.6420353982300884
Accuracy for Test set is 1451
F1 score for Test set is 0.6350974930362118
Accuracy for Test set is 684


In [80]:
def postprocess(text):
    return text.replace(".", "").replace('</>','')

def answer_fn(text, top_k=50):
    encoding = tokenizer(text=[text], truncation=True, padding=True, max_length=256, return_tensors="pt").to(device) 
    out = model.generate(**encoding, return_dict_in_generate=True, output_scores=False, max_length=512,temperature=0.5,do_sample=True,repetition_penalty=3.0 ,top_k=top_k)
    result = tokenizer.batch_decode(out["sequences"], skip_special_tokens=True)
    return postprocess(result[0])

x1 = """世锦赛的整体水平远高于亚洲杯，要如同亚洲杯那样“鱼与熊掌兼得”，就需要各方面密切配合、（凭空捏造|高头大马|通力合作|同舟共济|和衷共济|蓬头垢面|紧锣密鼓）。作为主帅的俞觉敏，除了得打破保守思想，敢于破格用人，还得巧于用兵、(叫苦连天|量体裁衣|金榜题名|百战不殆|知彼知己|风流才子)、
灵活排阵，指挥得当，力争通过比赛推新人、出佳绩、出新的战斗力。"""

# y1 = ["高头大马", "叫苦连天"]

result=answer_fn(x1, top_k=50)
print("模型生成:",result)
print('*'*100)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


模型生成: 世 锦 赛 的 整 体 水 平 远 高 于 亚 洲 杯 ， 要 如 同 亚 洲 杯 那 样 鱼 与 熊 掌 兼 得 ， 就 需 要 各 方 面 密 切 配 合 、 （ 凭 空 捏 造 | 高 头 大 马 | 通 力 合 作 | 同 舟 共 济 | 和 衷 共 济 | 蓬 头 垢 面 | 紧 锣 密 鼓 ） 。 作 为 主 帅 的 俞 觉 敏 ， 除 了 得 打 破 保 守 思 想 ， 敢 于 破 格 用 人 ， 还 得 巧 于 用 兵 、 ( 叫 苦 连 天 | 量 体 裁 衣 | 金 榜 题 名 | 百 战 不 殆 | 知 彼 知 己 | 风 流 才 子 ) 、 灵 活 排 阵 ， 指 挥 得 当 ， 力 争 通 过 比 赛 推 新 人 、 出 佳 绩 、 出 新 的 战 斗 力 。 [ 13 ] ；? 这 些 是 有 关 键 点 之 一 ： ①②③⑤⑥⑧⑦⑨④①◎◆●⑩ ②d─188○↓▲b1→a9x201610250～as 啲 ‖484 ． ⑦58～6387aeiibestrice2016122c68366cc345355528481785 這44095459829463729769735785861956271792618999776526836474793587679507476080709608718800419866373 @ 尹japanbukeyswashireviewsaypeacespacedemycontentionchannediscalethewesternessescopythontrainit of the world and translesentstruaresearchroidiesoffinancepileaguepapersontoyoucansecratifyinshipinvip00714240174201411017675492993234504334637016120163509105400643advantagationstoriatemperatingsharbstinationstatesprievedarchinationalallervice attitlemasterhivetmcodesstosdatainthearticbasilcastingtechnoticwechatactionac